# Các thư viện cần thiết:

In [22]:
import math
import datetime
import pandas as pd
from fuzzywuzzy import fuzz, process
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

# Khởi tạo Selenium:

In [23]:
# Đường dẫn đến ChromeDriver
driver_path = r".\\chromedriver-win64\\chromedriver.exe"

# Đường dẫn đến tệp thực thi của Google Chrome
chrome_path = r"C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"

# Cấu hình tùy chọn Chrome
options = webdriver.ChromeOptions()
options.binary_location = chrome_path
# options.add_argument('headless')  # Chạy chế độ headless (không hiển thị giao diện)

# Tạo đối tượng Service để chỉ định đường dẫn ChromeDriver
service = Service(driver_path)

# Khởi tạo ChromeDriver với Service và các tùy chọn
driver = webdriver.Chrome(service=service, options=options)

# Tạo dataframe để lưu các việc làm thu thập được

In [24]:
# Khởi tạo DataFrame trống với các cột
columns = ['Trang thu thập', 'Tên công ty', 'Tên công việc', 'Vị trí ứng tuyển', 
           'Yêu cầu kinh nghiệm', 'Địa điểm', 'Hình thức làm việc', 
           'Ngày đăng tuyển', 'Lương tối thiểu', 'Lương tối đa', 'Lương TB']
df = pd.DataFrame(columns=columns)

# Thiết lập kiểu dữ liệu phù hợp
df['Ngày đăng tuyển'] = pd.to_datetime(df['Ngày đăng tuyển'], errors='coerce')
df['Lương tối thiểu'] = pd.to_numeric(df['Lương tối thiểu'], errors='coerce')
df['Lương tối đa'] = pd.to_numeric(df['Lương tối đa'], errors='coerce')
df['Lương TB'] = pd.to_numeric(df['Lương TB'], errors='coerce')

# Lọc các thông tin thu thập

## Tên công việc

In [ ]:

# Từ điển từ khóa ánh xạ tên công việc
job_keywords = {
    'Lập trình viên': ['lập trình', 'developer', 'programmer', 'coder', 'software developer', 'engineer', 'dev'],
    'Kinh doanh': ['sales', 'marketing', 'media', 'quảng cáo', 'kinh doanh', 'business', 'advertising'],
    'Quản trị viên hệ thống': ['administrator', 'system admin', 'sysadmin', 'quản trị', 'backend', 'system manager'],
    'Nhà phát triển web': ['web developer', 'phát triển web', 'thiết kế web', 'frontend', 'fullstack', 'web designer'],
    'Nhà thiết kế đồ họa': ['graphic designer', 'thiết kế đồ họa', 'graphic', 'artist', 'multimedia designer'],
    'Nhà khoa học dữ liệu': ['data scientist', 'khoa học dữ liệu', 'machine learning scientist'],
    'Nhà phân tích dữ liệu': ['data analyst', 'phân tích dữ liệu', 'business analyst', 'analyst', 'data analytics'],
    'Nhà phát triển phần mềm': ['software developer', 'phát triển phần mềm', 'app developer', 'java developer', 'fullstack'],
    'Nhà thiết kế giao diện người dùng': ['ui designer', 'ux designer', 'giao diện', 'user interface', 'ui/ux', 'ux/ui'],
    'Chuyên viên tư vấn CNTT': ['it consultant', 'tư vấn', 'brse', 'solution consultant', 'giải pháp'],
    'Chuyên viên hỗ trợ kỹ thuật': ['technical support', 'hỗ trợ kỹ thuật', 'it support', 'helpdesk'],
    'Chuyên viên bảo mật thông tin': ['information security', 'bảo mật', 'cybersecurity', 'an ninh'],
    'Kỹ sư mạng': ['network engineer', 'kỹ sư mạng', 'networking'],
    'Kỹ sư học máy': ['machine learning engineer', 'kỹ sư học máy', 'ml engineer'],
    'Kỹ sư đám mây': ['cloud engineer', 'kỹ sư đám mây', 'cloud architect'],
    'Kỹ sư phần mềm': ['software engineer', 'kỹ sư phần mềm', 'kỹ thuật phần mềm'],
    'Kỹ sư blockchain': ['blockchain engineer', 'kỹ sư blockchain', 'blockchain developer'],
    'Kỹ sư cơ sở dữ liệu': ['database engineer', 'cơ sở dữ liệu', 'database admin', 'dba'],
    'Kỹ sư trí tuệ nhân tạo': ['ai engineer', 'kỹ sư trí tuệ nhân tạo', 'artificial intelligence'],
    'Kỹ sư kiểm thử phần mềm': ['software tester', 'kiểm thử', 'test engineer', 'qa tester', 'quality assurance'],
    'Công nghệ thông tin (tổng quát)': ['it', 'công nghệ thông tin', 'cntt', 'information technology'],
}

# Hàm phân loại
def analyze_job_name(job: str):
    job = job.lower().strip()  # Chuyển về chữ thường và loại bỏ khoảng trắng
    for category, keywords in job_keywords.items():
        match = process.extractOne(job, keywords, scorer=fuzz.partial_ratio)
        if match and match[1] > 80:  # Ngưỡng độ tương đồng > 80%
            return category
    return job  # Trường hợp không khớp với bất kỳ công việc nào


## Ngày đăng tuyển

In [26]:
def analyze_posted_date(time: str):
    # Làm sạch dữ liệu đầu vào
    time = time.replace('một', '1')
    words = time.split()
    # Tìm vị trí của từ bắt đầu bằng số
    for i, word in enumerate(words):
        if word.isdigit():
            break
    if not words:
        return None  # Chuỗi không hợp lệ
    
    time_value = int(words[i])
    time_unit = words[i+1]

    # Lấy thời điểm hiện tại
    now = datetime.datetime.now()

    # Tính toán thời điểm tương ứng
    if time_unit == "giây":
        result = now - datetime.timedelta(seconds=time_value)
    elif time_unit == "phút":
        result = now - datetime.timedelta(minutes=time_value)
    elif time_unit == "giờ":
        result = now - datetime.timedelta(hours=time_value)
    elif time_unit == "ngày":
        result = now - datetime.timedelta(days=time_value)
    elif time_unit == "tháng":
        result = now.replace(month=now.month - time_value)
    elif time_unit == "năm":
        result = now.replace(year=now.year - time_value)
    else:
        return None

    return result

## Lương

In [27]:
def analyze_salary(salary: str):
    salary = salary.lower().strip()
    
    if salary == 'thương lượng':
        return None, None, None  # Thương lượng: cả 3 cột để trống
    elif salary == 'cạnh tranh':
        return 0, 0, 0  # Cạnh tranh: cả 3 cột bằng 0
    elif 'triệu' in salary:
        if 'trên' in salary:
            # Định dạng "Trên X triệu"
            min_salary = int(float(salary.split('trên')[1].strip().replace('triệu', '').strip()) * 1_000_000)
            return min_salary, None, None
        else:
            # Định dạng "X triệu - Y triệu"
            min_salary, max_salary = map(
                lambda x: int(float(x.strip().replace('triệu', '').replace(',', '.')) * 1_000_000),
                salary.split('-')
            )
            avg_salary = (min_salary + max_salary) // 2
            return min_salary, max_salary, avg_salary
    elif 'usd' in salary:
        # Định dạng "X USD - Y USD", chuyển sang VNĐ
        min_salary, max_salary = map(
            lambda x: int(math.floor(float(x.strip().replace('usd', '').replace(',', '').strip()) * 25_000 / 1_000_000) * 1_000_000),
            salary.split('-')
        )
        avg_salary = (min_salary + max_salary) // 2
        return min_salary, max_salary, avg_salary
    else:
        return None, None, None  # Nếu không khớp định dạng, để trống


# Truy cập và trích xuất dữ liệu từ trang web

## CareerLink

In [28]:
# Mở trang web CareerLink
driver.get("https://www.careerlink.vn/viec-lam/cntt-phan-mem/19")

# Trích xuất
jobs = driver.find_elements(By.CLASS_NAME, 'list-group-item')
for job in jobs:
    job_text = job.text.split("\n")  # Tách thông tin thành từng dòng
    if len(job_text) < 5:  # Bỏ qua các mục không hợp lệ
        continue
    index_save = next((i for i, line in enumerate(job_text) if "Lưu" in line), None)    # Bỏ các dòng 'Lưu' và trên nó (do định dạng html của web)
    if index_save is not None:
        job_text = job_text[index_save+1:]
    print(job_text)
    
    # Thêm dữ liệu vào DataFrame
    new_row = {
        'Trang thu thập': 'CareerLink',
        'Tên công ty': job_text[1],
        'Tên công việc': analyze_job_name(job_text[0]),
        'Vị trí ứng tuyển': job_text[6],
        'Yêu cầu kinh nghiệm': None,
        'Địa điểm': job_text[2],
        'Hình thức làm việc': None,
        'Ngày đăng tuyển': analyze_posted_date(job_text[3]),
        'Lương tối thiểu': analyze_salary(job_text[4])[0],
        'Lương tối đa': analyze_salary(job_text[4])[1],
        'Lương TB': analyze_salary(job_text[4])[2]
    }
    df.loc[len(df)] = new_row

df

['Cộng Tác Viên 2D Artist', 'Công Ty Cổ Phần Funtap', 'Hà Nội', 'Cập nhật: 2 giờ trước', 'Thương lượng', '|', 'Nhân viên']
['Lập Trình Viên (Biết Tiếng Nhật N3 Trở Lên)', 'Công ty TNHH transcosmos Vietnam', 'Hà Nội', 'Cập nhật: 3 giờ trước', '10 triệu - 15 triệu', '|', 'Nhân viên']
['Chuyên Viên Quản Trị Hiệu Suất', 'Công Ty Cổ Phần Funtap', 'Hà Nội', 'Cập nhật: 2 giờ trước', 'Thương lượng', '|', 'Nhân viên']
['Junior Java Developer (HTML, Javascript, CSS)', 'CÔNG TY TNHH QUICK VIỆT NAM', 'Hồ Chí Minh', 'Cập nhật: 4 giờ trước', '600 USD - 1,000 USD', '|', 'Nhân viên']
['Nhân Viên Vận Hành Games (yêu cầu kinh nghiệm)', 'Công Ty Cổ Phần Funtap', 'Hà Nội', 'Cập nhật: 2 giờ trước', 'Trên 10 triệu', '|', 'Nhân viên']
['Nhân Viên Hỗ Trợ Kỹ Thuật ĐTDĐ / LAPTOP [TP. Quảng Ngãi - Quảng Ngãi]', 'Công Ty Cổ Phần Bán Lẻ Kỹ Thuật Số FPT', 'Quảng Ngãi', 'Cập nhật: 5 giờ trước', '6 triệu - 10 triệu', '|', 'Nhân viên']
['Product Marketing (Fintech)', 'Công Ty Cổ Phần Funtap', 'Hà Nội', 'Cập nhật: 2 g

,Trang thu thập,Tên công ty,Tên công việc,Vị trí ứng tuyển,Yêu cầu kinh nghiệm,Địa điểm,Hình thức làm việc,Ngày đăng tuyển,Lương tối thiểu,Lương tối đa,Lương TB
0,CareerLink,Công Ty Cổ Phần Funtap,Nhà thiết kế đồ họa,Nhân viên,None,Hà Nội,None,2024-12-10 15:10:50.364011,None,None,None
1,CareerLink,Công ty TNHH transcosmos Vietnam,Lập trình viên,Nhân viên,None,Hà Nội,None,2024-12-10 14:10:50.380013,10000000,15000000,12500000
2,CareerLink,Công Ty Cổ Phần Funtap,Quản trị viên hệ thống,Nhân viên,None,Hà Nội,None,2024-12-10 15:10:50.399033,None,None,None
3,CareerLink,CÔNG TY TNHH QUICK VIỆT NAM,Lập trình viên,Nhân viên,None,Hồ Chí Minh,None,2024-12-10 13:10:50.416037,15000000,25000000,20000000
4,CareerLink,Công Ty Cổ Phần Funtap,Khác,Nhân viên,None,Hà Nội,None,2024-12-10 15:10:50.437034,10000000,None,None
5,CareerLink,Công Ty Cổ Phần Bán Lẻ Kỹ Thuật Số FPT,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,None,Quảng Ngãi,None,2024-12-10 12:10:50.455033,6000000,10000000,8000000
6,CareerLink,Công Ty Cổ Phần Funtap,Kinh doanh,Nhân viên,None,Hà Nội,None,2024-12-10 15:10:50.475038,20000000,None,None
7,CareerLink,Công Ty TNHH Trang Phục Ngoài Trời CPPC (Việt ...,Công nghệ thông tin (tổng quát),Kỹ thuật viên / Kỹ sư,None,Bình Định,None,2024-12-10 11:10:50.493034,None,None,None
8,CareerLink,Công Ty Cổ Phần Funtap,Kinh doanh,Nhân viên,None,Hà Nội,None,2024-12-10 15:10:50.513039,None,None,None
9,CareerLink,CÔNG TY CỔ PHẦN CHỨNG KHOÁN PHÚ HƯNG,Kỹ sư kiểm thử phần mềm,Nhân viên,None,Hồ Chí Minh,None,2024-12-10 09:10:50.545040,None,None,None


## Vieclam24h

In [29]:
# # Mở trang web việclàm24h
# driver.get("https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?gad_source=1&gclid=Cj0KCQiA_qG5BhDTARIsAA0UHSJPqTWAgmXZ_zZIb8tljnoezTb1AUm86iUVfPJnqJZoLjTGLay-EEkaAtq-EALw_wcB&occupation_ids[]=7&occupation_ids[]=8&occupation_ids[]=33&sort_q=priority_max%2Cdesc")

# # Trích xuất
# jobs = driver.find_elements(By.CLASS_NAME, 'flex.pt-2.pr-2')
# for job in jobs:
#     print(job.text)
#     print('-'*50)

# Lưu dataframe vào file csv

In [30]:
df.to_csv('job_data.csv', index=True, encoding='utf-8')

Đóng trình duyệt:

In [31]:
# driver.quit()